IMPORT THE INPUT FILE

In [2]:
# Import the Google Colab drive lib
from google.colab import drive
drive.mount('/content/drive')

# Read the input file (Thermotoga petrophila genome)
input_path = open('/content/drive/My Drive/UFRN/10 - Cursos online/Finding Hidden Messages in DNA (Bioinformatics I)/1 - Where in the Genome Does Replication Begin/Thermotoga_petrophila.txt', 'r')
input = input_path.read()

# Print the input
print(input)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
aactctatacctcctttttgtcgaatttgtgtgatttatagagaaaatcttattaactgaaactaaaatggtaggtttggtggtaggttttgtgtacattttgtagtatctgatttttaattacataccgtatattgtattaaattgacgaacaattgcatggaattgaatatatgcaaaacaaacctaccaccaaactctgtattgaccattttaggacaacttcagggtggtaggtttctgaagctctcatcaatagactattttagtctttacaaacaatattaccgttcagattcaagattctacaacgctgttttaatgggcgttgcagaaaacttaccacctaaaatccagtatccaagccgatttcagagaaacctaccacttacctaccacttacctaccacccgggtggtaagttgcagacattattaaaaacctcatcagaagcttgttcaaaaatttcaatactcgaaacctaccacctgcgtcccctattatttactactactaataatagcagtataattgatctga


THE CODE LOGIC BEGINS HERE!

In [3]:
# ======================================
# Prefix function
# ======================================
def Prefix(input_):
  return input_[0:(len(input_) - 1)].upper()

# ======================================
# LastSymbol function
# ======================================
def LastSymbol(input_):
  return input_[-1].upper()

# ======================================
# SymbolToNumber function
# ======================================
def SymbolToNumber(input_):
  input_ = input_.upper()
  result = ""

  # Iterate over the input string
  for idx, chr in enumerate(input_):
    # Check the lexicographic order
    switcher = {
      "A": "0",
      "C": "1",
      "G": "2",
      "T": "3"
    }
    current_result = switcher.get(input_[idx], "Invalid base")

    # Verify if the current base is valid!
    if (current_result == "Invalid base"):
      return -1
    else:
      result += current_result

  # Return the final result
  return int(result)

# ======================================
# NumberToSymbol function
# ======================================
def NumberToSymbol(input_):
  input_ = str(input_)
  result = ""

  # Iterate over the input number
  for idx, chr in enumerate(input_):
    # Check the lexicographic order
    switcher = {
      "0": "A",
      "1": "C",
      "2": "G",
      "3": "T"
    }
    current_result = switcher.get(input_[idx], "Invalid number")

    # Verify if the current base is valid!
    if (current_result == "Invalid number"):
      return -1
    else:
      result += current_result

  # Return the final result
  return result

# ======================================
# NumberToPattern function
# ======================================
def NumberToPattern(index_, k_):
  # Check if the input k-mer has size 1
  if k_ == 1:
    return NumberToSymbol(index_)

  # Get the Quotient(index, 4)
  prefixIndex = index_ // 4

  # Get the Remainder(index, 4)
  r = index_ % 4

  # Retrieve the equivalent symbol from remainder
  symbol = NumberToSymbol(r)

  # Recursive call with a smaller k-mer
  prefixPattern = NumberToPattern(prefixIndex, k_ - 1)

  # Return concatenation of PrefixPattern with symbol
  return str(prefixPattern) + str(symbol)

# ======================================
# PatternToNumber function
# ======================================
def PatternToNumber(input_):
  # Check if the input Pattern contains at least 1 char
  if len(input_) == 0:
    return 0

  # Reduce the k-mer size until get the prefix and suffix positions
  return 4 * PatternToNumber(Prefix(input_)) + SymbolToNumber(LastSymbol(input_))

In [4]:
# ======================================
# ComputingFrequencies function
# ======================================
def ComputingFrequencies(input_, k_):
  # Declare the frequencyArray
  frequencyArray = []

  # Calculate the loop limits
  loopLimit1 = (4**k_)
  loopLimit2 = len(input_) - (k_-1)

  # Loop 1: Initialize the frequencyArray with zeros
  for idx in range(loopLimit1):
    frequencyArray.append(0)

  # Loop 2: Iterate over the |Text| − k
  for idx in range(loopLimit2):
    current_pattern = input_[idx:(idx + k_)]
    current_number = PatternToNumber(current_pattern)
    frequencyArray[current_number] = frequencyArray[current_number] + 1

  # Return the frequency array
  return frequencyArray

In [5]:
# ============================================================================
# FasterFrequentWords function
# 
# FrequentWords (FW) is (O((|Text|^2) * k)) 
# FasterFrequentWords (FFW) is (O((|Text|*k) + ((4^k) * k))).
#
# For all values of k such that 4^k  < |Text|^2 - |Text|, FFW will always be 
# faster (lesser steps to execute) than FW.
#
# At  4^k  = |Text|^2 - |Text| they will be almost equally efficient.
#
# For all values of  k such that 4^k  > |Text|^2 - |Text|, FFW will always be 
# slower (more steps to execute) than FW.
# ============================================================================
def FasterFrequentWords(input_, k_):
  # Declare the frequentPatterns
  frequentPatterns = []

  # Fill the frequencyArraya and get its max value
  frequencyArray = ComputingFrequencies(input_, k_)
  maxCount = max(frequencyArray)

  # Calculate the loop limits
  loopLimit1 = (4**k_)

  # Loop 1: Initialize the frequencyArray with zeros
  for idx in range(loopLimit1):
    if frequencyArray[idx] == maxCount:
      current_pattern = input_[idx:(idx + k_)]
      frequentPatterns.append(current_pattern)

  # Return the frequentPatterns array with blank patterns removed
  return list(filter(None, frequentPatterns))

In [11]:
# ======================================
# Test section
# ======================================
# Test 1
output1 = FasterFrequentWords("ACGCGGCTCTGAAA", 2)
print("(ACGCGGCTCTGAAA, 2) = " + str(output1))

# Test 2
output2 = FasterFrequentWords("ACGCGGCTCTGAAA", 3)
print("(ACGCGGCTCTGAAA, 3) = " + str(output2))

# Test 3
output3 = FasterFrequentWords("ACGCGGCTCTGAAA", 4)
print("(ACGCGGCTCTGAAA, 4) = " + str(output3))

# Test 4
output4 = FasterFrequentWords("ACGCGGCTCTGAAA", 5)
print("(ACGCGGCTCTGAAA, 5) = " + str(output4))

# Test 5
output5 = FasterFrequentWords("ACGCGGCTCTGAAA", 6)
print("(ACGCGGCTCTGAAA, 6) = " + str(output5))

# Test 6 (Thermotoga petrophila genome)
output6 = FasterFrequentWords(input.upper(), 9)
print("(Thermotoga petrophila, 9) = " + str(output6))

(ACGCGGCTCTGAAA, 2) = ['AC', 'CT', 'TC', 'TG']
(ACGCGGCTCTGAAA, 3) = ['ACG', 'CTC']
(ACGCGGCTCTGAAA, 4) = []
(ACGCGGCTCTGAAA, 5) = []
(ACGCGGCTCTGAAA, 6) = []
(Thermotoga petrophila, 9) = []
